<a href="https://colab.research.google.com/github/MLRG-CEFET-RJ/attention-mechanism/blob/main/dados_radar_extremo_3h.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wradlib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import wradlib as wrl
import os
import numpy as np
import tarfile
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from glob import glob
from IPython.display import HTML

In [5]:
path = '/content/drive/MyDrive/Colab Notebooks'
tar_filename = os.path.join(path, 'radar 3hrs.tar')
vol_pattern = os.path.join(path, '*.vol')
with tarfile.open(tar_filename, 'r') as tar_file:
    tar_file.extractall(path)

In [6]:
rainbow_files = glob(vol_pattern)

In [7]:
def read_radar_data(filename):
  data = wrl.io.read_rainbow(filename)
  return data

In [8]:
def get_sensor_info(radar):
  sensor_info = radar['volume']['sensorinfo']
  name = sensor_info['@name']
  lat = sensor_info['lat']
  lon = sensor_info['lon']
  return (name, lat, lon)

In [9]:
def get_slices(radar):
  return radar['volume']['scan']['slice']

In [10]:
def get_one_slice(radar, num=0):
  return get_slices(radar)[num]

In [11]:
def get_azimuthal_data(slice_):
  ray_info = slice_['slicedata']['rayinfo']
  azi_data = ray_info['data']
  azi_depth = float(ray_info['@depth'])
  azi_range = float(ray_info['@rays'])
  elevation_angle = float(slice_['posangle'])
  azi_data = (azi_data * azi_range / 2 ** azi_depth)
  return {
      'azi_data': azi_data,
      'azi_depth': azi_depth,
      'azi_range': azi_range,
      'elevation_angle': elevation_angle
  }

In [12]:
def define_range(slice_):
  stoprange = float(slice_['stoprange'])
  rangestep = float(slice_['rangestep'])
  range = np.arange(0, stoprange, rangestep)
  return range

In [13]:
def get_reflectivity(slice_, num=0):
  rawdata = slice_['slicedata']['rawdata'][num]
  data = rawdata['data']
  data_depth = float(rawdata['@depth'])
  data_min = float(rawdata['@min'])
  data_max = float(rawdata['@max'])
  data = data_min + data * (data_max - data_min) / 2 ** data_depth
  return data

In [14]:
def plot_radar_data(reflectivity, range, azi_data, elevation_angle, sensor_name, 
                    lat, lon):
  ax, pm = wrl.vis.plot_ppi(reflectivity, r=range, az=azi_data, fig=fig);
  title = f'{sensor_name}\n day: 14/02/2018, time: 19:00 until 23:55\n elevation angle: {elevation_angle} \n{lat}N {lon}E'
  plt_title = plt.title(title, fontsize=14);
  plt_title.set_y(1.05);
  cbar = plt.gcf().colorbar(pm, pad=0.075, shrink=0.75);
  cbar.set_label('reflectivity [dBz]')
  ax.set_xlabel('x_range [km]');
  ax.set_ylabel('y_range [km]');
  return ax

In [15]:
def animate(i, sensor_name, lat, lon):
  radar_data = read_radar_data(rainbow_files[i])
  elevation_slice = get_one_slice(radar_data)
  azi_dict = get_azimuthal_data(elevation_slice)
  range = define_range(elevation_slice)
  reflectivity = get_reflectivity(elevation_slice)
  fig.clear()
  image = plot_radar_data(reflectivity, range, azi_dict['azi_data'], 
                       azi_dict['elevation_angle'], sensor_name, lat, lon)
  return image

In [16]:
sensor_name, lat, lon = get_sensor_info(read_radar_data(rainbow_files[0]))

In [17]:
%%capture
fig = plt.figure(figsize=(8,8));
anim = FuncAnimation(fig, animate, frames=len(rainbow_files), interval=250, 
                     fargs=(sensor_name, lat, lon), blit=False);
animation_control = HTML(anim.to_jshtml())

KeyError: ignored

In [18]:
animation_control

NameError: ignored